In [1]:
import random

def custom_boolean(chance_of_truth=50):
    return random.choices([True, False], weights=[chance_of_truth, 100 - chance_of_truth])[0]

In [2]:
from faker import Faker
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib.units import inch

In [3]:


fake = Faker()
styles = getSampleStyleSheet()

def generate_account_number():
    chance = custom_boolean()
    account_number = "0230" if chance else ""
    
    if chance:
        account_number += "."
    account_number += fake.numerify(text='#####')
    if chance:
        account_number += "."
        account_number += fake.bothify(text='??#')
    return account_number

def generate_email(email_number):
    sender_name = fake.name()
    sender_team = fake.job() + " Team"
    sender_bank = fake.company() + " Financial"
    recipient_team = "Operations Group, " + fake.company()
    trade_date = fake.date_between(start_date="-30d", end_date="today").strftime('%Y-%m-%d')
    settlement_date = fake.date_between(start_date="today", end_date="+30d").strftime('%Y-%m-%d')
    response_deadline = fake.date_between(start_date="+1d", end_date="+7d").strftime('%Y-%m-%d')

    email_bodies = [
        f"This is to inform you that the trade detailed below has not settled as scheduled. We need your immediate attention to resolve this.",
        f"We are writing to notify you of an unsettled trade. Kindly look into the details below and provide an update on the settlement.",
        f"We are contacting you regarding a trade that is still unsettled. Please review the details below and provide an update on the settlement status as soon as possible.",
        f"This email is to inform you of a delay in the settlement of the trade detailed below. We request your prompt review and an update on the settlement status.",
        f"We are following up on the unsettled trade detailed below. Kindly review the information and provide an update on the settlement status at your earliest convenience.",
        f"Please be advised that the following trade is currently unsettled. Your attention to this matter and an update on the settlement status would be greatly appreciated.",
        f"This is a reminder regarding the unsettled trade detailed below. We kindly request that you review the details and provide an update on the settlement status as soon as possible.",
        f"We are writing to you with urgency concerning the unsettled trade detailed below. Your immediate review and an update on the settlement status are required.",
        f"We need your action on the unsettled trade detailed below. Please review the details and provide an update on the settlement status as soon as possible.",
        f"We are requesting an update on the unsettled trade detailed below. Kindly review the information and provide the settlement status at your earliest convenience.",
    ]

    email_body = email_bodies[email_number % len(email_bodies)] # Cycle through email bodies

    return {
        "Subject": f"Unsettled Trade Notification - {fake.word().capitalize()}",
        "From": f"{sender_name}, {sender_team}, {sender_bank}",
        "To": recipient_team,
        "CC": "Trade Services, Global Clearing Corp.", # Static CC
        "Body": f"Dear {recipient_team},\n\n{email_body}\n\nTrade Details: [See Table Below]\n\nIssue: Settlement was due on {settlement_date}, but the transaction remains incomplete. Please investigate the cause of this delay and advise on the steps needed to finalize the settlement.\n\nRequested Action:\n\n1.  Provide the current status of this trade.\n2.  Inform us of any actions required from our side.\n3.  Report any discrepancies that need to be addressed.\n\nYour swift response is crucial. Please acknowledge this email and provide an update by {response_deadline}.\n\nBest regards,\n\n{sender_name}\n{sender_team}\n{sender_bank}\n{fake.street_address()}, {fake.city()}, {fake.country()}\nEmail: {fake.email()}\nPhone: {fake.phone_number()}\nFax: {fake.phone_number()}\nCustodian Account: {generate_account_number()}",
        "Trade_Details_Table": [
            ["Trade Date", "Quantity", "ISIN", "Custodian Account"],
            [fake.date_between(start_date="-60d", end_date="-30d").strftime('%Y-%m-%d'), fake.random_int(min=100, max=1000), fake.bothify(text='US##########'), generate_account_number()],
            [fake.date_between(start_date="-60d", end_date="-30d").strftime('%Y-%m-%d'), fake.random_int(min=100, max=1000), fake.bothify(text='GB##########'), generate_account_number()],
            [fake.date_between(start_date="-60d", end_date="-30d").strftime('%Y-%m-%d'), fake.random_int(min=100, max=1000), fake.bothify(text='FR##########'), generate_account_number()],
            [fake.date_between(start_date="-60d", end_date="-30d").strftime('%Y-%m-%d'), fake.random_int(min=100, max=1000), fake.bothify(text='DE##########'), generate_account_number()],
            [fake.date_between(start_date="-60d", end_date="-30d").strftime('%Y-%m-%d'), fake.random_int(min=100, max=1000), fake.bothify(text='JP##########'), generate_account_number()],
        ]
    }

def build_email_elements(email_data):
    elements = []
    elements.append(Paragraph(email_data["Subject"], styles['h1']))
    elements.append(Spacer(1, 0.2*inch))
    elements.append(Paragraph(email_data["From"], styles['Normal']))
    elements.append(Paragraph(f"To: {email_data['To']}", styles['Normal']))
    elements.append(Paragraph(f"CC: {email_data['CC']}", styles['Normal']))
    elements.append(Spacer(1, 0.2*inch))
    elements.append(Paragraph(email_data["Body"], styles['Normal']))
    elements.append(Spacer(1, 0.2*inch))

    table = Table(email_data["Trade_Details_Table"])
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    elements.append(table)
    elements.append(Spacer(1, 0.2*inch))

    return elements

def generate_emails_pdf(num_emails=10):


    for i in range(num_emails):

        doc_name = f"unsettled_trade_email_{i}.pdf"
        doc = SimpleDocTemplate(doc_name, pagesize=letter)
        all_elements = []
        email_data = generate_email(i)
        email_elements = build_email_elements(email_data)
        all_elements.extend(email_elements)
        all_elements.append(Spacer(1, 0.5 * inch))  # Add space between emails
        doc.build(all_elements)
        print(f"PDF {doc_name} generated successfully.")

if __name__ == '__main__':
    generate_emails_pdf()

PDF unsettled_trade_email_0.pdf generated successfully.
PDF unsettled_trade_email_1.pdf generated successfully.
PDF unsettled_trade_email_2.pdf generated successfully.
PDF unsettled_trade_email_3.pdf generated successfully.
PDF unsettled_trade_email_4.pdf generated successfully.
PDF unsettled_trade_email_5.pdf generated successfully.
PDF unsettled_trade_email_6.pdf generated successfully.
PDF unsettled_trade_email_7.pdf generated successfully.
PDF unsettled_trade_email_8.pdf generated successfully.
PDF unsettled_trade_email_9.pdf generated successfully.
